In [1]:
import numpy as np
import urllib
import scipy.optimize
from scipy.cluster.vq import *
import random
from sklearn.decomposition import PCA
from collections import defaultdict
import matplotlib.pyplot as plt
import gzip
from collections import defaultdict

In [2]:
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [3]:
allRatings = []
allHelpful = []
#itemCount = defaultdict(int)
userItemPair = defaultdict(list) # this will store for each user what item they bought
itemUserPair = defaultdict(list) # set of users who purchased the item i
ratingMatrix = defaultdict(dict) # dictionary inside dictionary to build R matrix 
wordCount = defaultdict(int) # store how many reviews has certain text length (length: how many)
wordRatingPair = defaultdict(list) # store (wordlength, rating pair)
ratingWordPair = defaultdict(list) # store (rating, wordlength pair)
#helpfulMatrix = defaultdict(list) # the acutal helpfulness
alpha = 0.0
dataSetSize = 0.0
label = [] # remember the nhelpful/outOf
X = []

for l in readGz("homework3/train.json.gz"):
    length = len(l['reviewText'].split())
    rating = l['rating']
    X.append([1,length,rating])
    label.append(l['helpful']['nHelpful'] * 1.0 / l['helpful']['outOf'])
    wordCount[length] += 1
    # length -> (rating,helpful)
    wordRatingPair[length].append((rating,l['helpful']))
    # rating -> (length,helpful)
    ratingWordPair[rating].append((length,l['helpful']))
    dataSetSize = dataSetSize + 1
    # for part 2 fit predictor of user and item
    user,item = l['reviewerID'],l['itemID']
    allRatings.append(rating) 
    allHelpful.append(l['helpful'])
    ratingMatrix[user].update({item: rating}) # build R matrix
    #helpfulMatrix[length].append((rating,l['helpful']))
    userItemPair[user].append(item)
    itemUserPair[item].append(user)
print "dataset size is ", dataSetSize

dataset size is  100000.0


In [83]:
s = 0.0
for user in userItemPair:
    itemList = userItemPair[user]
    for i in itemList:
        s = s + ratingMatrix[user][i]
s_other = sum(allRatings) 

print s
print s_other

360328.0
360335.0


In [79]:
averageRate = sum([x for x in allRatings]) * 1.0 / len(allRatings)
print "simple predictor alpha is ", averageRate

simple predictor alpha is  3.60335


In [69]:
# calculate the MSE
testRating = []
with open("homework3/labeled_Rating.txt") as f:
    for line in f:
        line = line.split()
        testRating.append(float(line[2]))
        
# calculate MSE
mean = 3.60335
error = 0.0

for rating in testRating:
    error = error + pow((rating - mean),2)
    
print "MSE for simple predictor is ", error * 1.0/len(testRating)

MSE for simple predictor is  2.6178318925


In [18]:
# part 2 
beta_i = defaultdict(float) # indexed by item id
beta_u = defaultdict(float) # indexed by user id 

# intialize the bias for each user and item to be zero
for i in itemUserPair:
    beta_i[i] = 0.0

for i in userItemPair:
    beta_u[i] = 0.0

#print userItemPair['U416283446']

oldmse = 0.0
newmse = 1000.0
optimization = 0.0
prev_optimization = optimization
new_optimization = 1000.0

alpha = 0.0
ratingList = []
# for every user in the dataset
for user in userItemPair:
    # get the list of item the user bought
    itemList= userItemPair[user]
    # for every item the user bought
    for item in itemList:
        ratingList.append(ratingMatrix[user][item])
        #alpha = alpha + ratingMatrix[user][item]

print "Alpha is !!! ", alpha
# the object should go down every iteration
#while abs(prev_optimization - new_optimization) > 0.0001 * prev_optimization:
#while abs(prev_optimization - new_optimization) > 0.00001:
while prev_optimization != new_optimization:
#while abs(oldmse - newmse) > 0.0001 * oldmse:
    alpha = 0.0
    # for every user in the dataset
    for user in userItemPair:
        # get the list of item the user bought
        itemList= userItemPair[user]
        # for every item the user bought
        for item in itemList:
            alpha = alpha + ratingMatrix[user][item] - (beta_u[user] + beta_i[item])
    alpha = alpha/dataSetSize
    print "alpha is ",alpha
    # update beta_u
    for user in userItemPair:
        itemList = userItemPair[user]
        updateValue = 0.0
        for item in itemList:
            updateValue = updateValue + ratingMatrix[user][item] - (alpha + beta_i[item])
        updateValue = updateValue * 1.0 / (1.0+ len(itemList))
        beta_u[user] = updateValue
    #print "desired user is ", beta_u['U566105319']
    # update beta_i
    for item in itemUserPair:
        # set of user who bought the item
        userList = itemUserPair[item]
        updateValue = 0.0
        for user in userList:
            updateValue = updateValue + ratingMatrix[user][item]-(alpha+ beta_u[user])
        updateValue = updateValue * 1.0/(1.0 + len(userList))
        beta_i[item] = updateValue
    #print "desired item is ",beta_i['I102776733']
    # calculate MSE
    mse = 0.0
    for user in userItemPair:
        itemList = userItemPair[user]
        for item in itemList:
            mse = mse + pow((ratingMatrix[user][item] - (alpha + beta_i[item] + beta_u[user])),2)      
    #mse = mse*1.0/dataSetSize
    optimization = 0.0
    for user in beta_u:
        optimization = beta_u[user]**2
    for item in beta_i:
        optimization = beta_i[item]**2
    optimization = mse + optimization
    prev_optimization = new_optimization
    new_optimization = optimization
    oldmse = newmse
    newmse = mse
    #print "old mse is ", oldmse
    #print "new mse is ", newmse
    print "new optimization is ", new_optimization

print "alpha is ", alpha
print "item bias is ", beta_i['I102776733']
print "user bias is ", beta_u['U566105319']

Alpha is !!!  0.0
alpha is  3.60328
new optimization is  54973.5808094
alpha is  3.5985194073
new optimization is  55023.7818797
alpha is  3.59288168054
new optimization is  55188.3152969
alpha is  3.58737371373
new optimization is  55267.2373664
alpha is  3.58223858624
new optimization is  55300.5100083
alpha is  3.57750741975
new optimization is  55313.9942317
alpha is  3.57315584256
new optimization is  55319.1254344
alpha is  3.56914905612
new optimization is  55320.7211764
alpha is  3.56545400097
new optimization is  55320.8260286
alpha is  3.56204181154
new optimization is  55320.323046
alpha is  3.55888761326
new optimization is  55319.5986234
alpha is  3.55596978273
new optimization is  55318.821866
alpha is  3.55326925532
new optimization is  55318.0650449
alpha is  3.55076899596
new optimization is  55317.3568194
alpha is  3.54845361895
new optimization is  55316.7062265
alpha is  3.54630911777
new optimization is  55316.1136491
alpha is  3.54432267002
new optimization is  55

In [21]:
labeled_rating = open("homework3/labeled_Rating.txt", 'r')
sum_value = 0.0
index = 0
for line in labeled_rating:
    index = index + 1
    line = line.split()
    sum_value = sum_value + (alpha + beta_u[line[0]] + beta_i[line[1]] - (float(line[2])))**2
print "MSE on test set is ", sum_value / index
labeled_rating.close()

MSE on test set is  2.19890447767


In [ ]:
# part 2 with double counting
beta_i = defaultdict(float) # indexed by item id
beta_u = defaultdict(float) # indexed by user id 

# intialize the bias for each user and item to be zero
for i in itemUserPair:
    beta_i[i] = 0.0

for i in userItemPair:
    beta_u[i] = 0.0

#print userItemPair['U416283446']

oldmse = 0.0
newmse = 1000.0
optimization = 0.0
prev_optimization = optimization
new_optimization = 1000.0

alpha = 0.0
ratingList = []
# for every user in the dataset
for user in userItemPair:
    # get the list of item the user bought
    itemList= userItemPair[user]
    # for every item the user bought
    for item in itemList:
        ratingList.append(ratingMatrix[user][item])
        #alpha = alpha + ratingMatrix[user][item]

print "Alpha is !!! ", alpha
# the object should go down every iteration
#while abs(prev_optimization - new_optimization) > 0.0001 * prev_optimization:
while abs(prev_optimization - new_optimization) > 0.0001:
#while abs(oldmse - newmse) > 0.0001 * oldmse:
    alpha = 0.0
    # for every user in the dataset
    for user in userItemPair:
        # get the list of item the user bought
        itemList= userItemPair[user]
        # for every item the user bought
        for item in itemList:
            alpha = alpha + ratingMatrix[user][item] - (beta_u[user] + beta_i[item])
    alpha = alpha/dataSetSize
    print "alpha is ",alpha
    # update beta_u
    for user in userItemPair:
        itemList = userItemPair[user]
        updateValue = 0.0
        for item in itemList:
            updateValue = updateValue + ratingMatrix[user][item] - (alpha + beta_i[item])
        updateValue = updateValue * 1.0 / (1.0+ len(itemList))
        beta_u[user] = updateValue
    #print "desired user is ", beta_u['U566105319']
    # update beta_i
    for item in itemUserPair:
        # set of user who bought the item
        userList = itemUserPair[item]
        updateValue = 0.0
        for user in userList:
            updateValue = updateValue + ratingMatrix[user][item]-(alpha+ beta_u[user])
        updateValue = updateValue * 1.0/(1.0 + len(userList))
        beta_i[item] = updateValue
    #print "desired item is ",beta_i['I102776733']
    # calculate MSE
    mse = 0.0
    for user in userItemPair:
        itemList = userItemPair[user]
        for item in itemList:
            mse = mse + pow((ratingMatrix[user][item] - (alpha + beta_i[item] + beta_u[user])),2)      
    #mse = mse*1.0/dataSetSize
    optimization = 0.0
    for user in beta_u:
        optimization = beta_u[user]**2
    for item in beta_i:
        optimization = beta_i[item]**2
    optimization = mse + optimization
    prev_optimization = new_optimization
    new_optimization = optimization
    oldmse = newmse
    newmse = mse
    #print "old mse is ", oldmse
    #print "new mse is ", newmse
    print "new optimization is ", new_optimization

print "alpha is ", alpha
print "item bias is ", beta_i['I102776733']
print "user bias is ", beta_u['U566105319']

In [5]:
# part 3 Jaccard Similarity
'''
Compute the Jaccard similarity between the users ‘U229891973’ and ‘U622491081’ in terms of the
sets of items they have reviewed in the training set.
'''
firstUser = "U229891973"
secondUser = "U622491081"

first_user= [] # record all the items first user reviewed
second_user = [] # record all the items the second user reviewed

first_user = userItemPair[firstUser]
second_user = userItemPair[secondUser]
print first_user
print second_user

def intersect(a, b):
    """ return the intersection of two lists """
    return list(set(a) & set(b))

def union(a, b):
    """ return the union of two lists """
    return list(set(a) | set(b))

similarity = len(intersect(first_user,second_user)) * 1.0/len(union(first_user, second_user))
print "jaccard similarity is ", similarity

['I919807826', 'I113307315', 'I623654409', 'I298464910']
['I113307315', 'I623654409', 'I298464910']
jaccard similarity is  0.75


In [11]:
'''
Find the user (or users) with the highest Jaccard similarity to ‘U622491081’ (if there are multiple
users with the same maximum similarity, list all of them)
'''
target_user = "U622491081"
target_user_item = userItemPair[target_user]
result = -1000.0
resultList = []
prev = 0.0

for user in userItemPair:
    similarity = 0.0
    itemList = userItemPair[user]
    similarity = len(intersect(target_user_item,itemList)) * 1.0/len(union(target_user_item,itemList))
    if len(resultList) != 0:
        prev = resultList[0][1] # get the previous best result
    # if similarity is better than the previous one
    if similarity > prev:
        # clear the previous highest jaccard similarity
        resultList = []
        resultList.append((user,similarity))
    elif similarity >= prev:
        resultList.append((user,similarity))    
print "highest Jaccard Similarity user ", resultList
print "how many user there are ", len(resultList)

user = []
for item in resultList:
    user.append(item[0])
    
print user

highest Jaccard Similarity user  [('U359587607', 1.0), ('U687939146', 1.0), ('U096951499', 1.0), ('U296575297', 1.0), ('U622491081', 1.0), ('U387971231', 1.0), ('U300899166', 1.0)]
how many user there are  7
['U359587607', 'U687939146', 'U096951499', 'U296575297', 'U622491081', 'U387971231', 'U300899166']


In [117]:
# part 4
alpha = 0.0
alpha = sum([x['nHelpful'] for x in allHelpful]) * 1.0 / sum([x['outOf'] for x in allHelpful])
print "alpha is ", alpha

alpha is  0.546807616187


In [119]:
labeled_helpful = open("homework3/labeled_Helpful.txt", 'r')
sum_value = 0.0
ASE = 0.0
index = 0
for line in labeled_helpful:
    index = index + 1
    line = line.split()
    ASE = ASE + abs(float(line[3]) - (alpha * float(line[2])))
    sum_value = sum_value + (float(line[3]) - (alpha * float(line[2])))**2
print "MSE on training set is ", sum_value / index
labeled_rating.close()
print "ASE on training set is ", ASE

MSE on training set is  75.8569915162
ASE on training set is  18336.6248542


In [8]:
# part 4c
beta_l = defaultdict(float) # indexed by word length
beta_r = defaultdict(float) # indexed by rating

# intialize the bias for each user and item to be zero
for i in wordRatingPair:
    beta_l[i] = 0.0

for i in ratingWordPair:
    beta_r[i] = 0.0

#print beta_l
print beta_r
    
oldmse = 0.0
newmse = 1000.0
alpha = 0.0

# the object should go down every iteration
while abs(oldmse - newmse) > 0.0001 * oldmse:
    alpha = 0.0
    index = 0.0
    # for every length in the dataset
    for length in wordRatingPair:
        # for that length, all the rating and helpful
        ratingList = wordRatingPair[length]
        #print ratingList
        for h in ratingList:
            rating = h[0]
            num = float(h[1]['nHelpful'])
            outOf = float(h[1]['outOf'])
            index = index + outOf
            alpha = alpha + (num - ((beta_l[length] + beta_r[rating]) * outOf))  
    alpha = alpha/index
    print "alpha is ",alpha
    #alpha = alpha/index
    # update beta_l
    # for every length in the data set
    index = 0.0
    for length in wordRatingPair:
        # for that length, all the rating and helpful list
        ratingList = wordRatingPair[length] # get a list of (rating,{helpful})
        updateValue = 0.0
        for h in ratingList:
            rating = h[0]
            num = float(h[1]['nHelpful'])
            outOf = float(h[1]['outOf'])
            index = index + outOf
            updateValue = updateValue + (num - ((alpha + beta_r[rating])*outOf))
        # doesn't have lambda
        updateValue = updateValue * 1.0 / index 
        beta_l[length] = updateValue
    print "update l done"
    #print "desired user is ", beta_u['U566105319']
    # update beta_r
    # for the same rating
    for rating in ratingWordPair:
        # all the word length for that rating
        wordLengthList = ratingWordPair[rating]
        # for each word length
        index = 0.0 # calcualte total outOf
        updateValue = 0.0
        for l in wordLengthList:
            length = l[0]
            num = l[1]['nHelpful']
            outOf = l[1]['outOf']
            index = index + outOf
            updateValue = updateValue + (num -(alpha+ beta_l[length])*outOf)
            # doesn't have lambda
        updateValue = updateValue * 1.0/index
        beta_r[rating] = updateValue
    print "updated r done"
    # calculate MSE
    mse = 0.0
    index = 0.0 
    for length in wordRatingPair:
        helpfulList = wordRatingPair[length] # get a list of (rating,{helpful})
        for h in helpfulList:
            rating = h[0]
            num = float(h[1]['nHelpful'])
            outOf = float(h[1]['outOf'])
            index = index + outOf
            mse = mse + pow((num - ((alpha + beta_l[length] + beta_r[rating])* outOf)),2)        
    mse = mse*1.0/index
    oldmse = newmse
    newmse = mse
    #print "old mse is ", oldmse
    print "new mse is ", newmse

# print "alpha is ", alpha
print beta_r

defaultdict(<type 'list'>, {1.0: [(148, {'nHelpful': 3, 'outOf': 3}), (20, {'nHelpful': 1, 'outOf': 1}), (27, {'nHelpful': 5, 'outOf': 9}), (69, {'nHelpful': 1, 'outOf': 3}), (51, {'nHelpful': 1, 'outOf': 1}), (158, {'nHelpful': 1, 'outOf': 1}), (88, {'nHelpful': 0, 'outOf': 1}), (32, {'nHelpful': 8, 'outOf': 33}), (1085, {'nHelpful': 6, 'outOf': 10}), (49, {'nHelpful': 1, 'outOf': 2}), (134, {'nHelpful': 0, 'outOf': 1}), (27, {'nHelpful': 1, 'outOf': 12}), (21, {'nHelpful': 0, 'outOf': 1}), (137, {'nHelpful': 2, 'outOf': 3}), (90, {'nHelpful': 0, 'outOf': 1}), (31, {'nHelpful': 1, 'outOf': 2}), (57, {'nHelpful': 0, 'outOf': 1}), (68, {'nHelpful': 2, 'outOf': 3}), (97, {'nHelpful': 5, 'outOf': 5}), (52, {'nHelpful': 2, 'outOf': 4}), (36, {'nHelpful': 1, 'outOf': 79}), (46, {'nHelpful': 5, 'outOf': 5}), (88, {'nHelpful': 2, 'outOf': 16}), (37, {'nHelpful': 0, 'outOf': 1}), (28, {'nHelpful': 3, 'outOf': 4}), (159, {'nHelpful': 10, 'outOf': 28}), (21, {'nHelpful': 4, 'outOf': 5}), (35, {'

In [45]:
# part 4c linera regression
#import numpy as np
#tmp = np.array(featureRating)
#print type(tmp)
#print featureLength
#X = np.concatenate((featureLength, tmp.T), axis=2)
#print X
theta,residuals,rank,s = np.linalg.lstsq(X, label)
print "theta is ", theta
alpha = theta[0]
beta1 = theta[1]
beta2 = theta[2]
print "alpha is ", alpha
print "beta 1 is ", beta1
print "beta 2 is ",beta2

# part 4d
# use helpful.train.json
testData = []
for l in readGz("homework3/helpful.json.gz"):
    length = len(l['reviewText'].split())
    rating = l['rating']
    outof = l['helpful']['outOf']
    testData.append([length,rating,outof])
#print testData

labeled_helpful = open("homework3/labeled_Helpful.txt", 'r')
sum_value = 0.0
ASE = 0.0
index = 0
for line in labeled_helpful:
    line = line.split()
    actualHelpful = float(line[3]) # the last index is the total number of helpful
    sum_value = sum_value + ((alpha+testData[index][0]*beta1 + testData[index][1] * beta2)*testData[index][2] - actualHelpful)**2
    ASE = ASE + abs(actualHelpful - ((alpha+testData[index][0]*beta1 + testData[index][1] * beta2)*testData[index][2]))
    index = index + 1
labeled_rating.close()
#print "ASE on training set is ", ASE
sum_value = sum_value/index
print "MSE is ", sum_value
print "ASE is ", ASE

theta is  [  3.45431358e-01   2.71683612e-04   4.39997995e-02]
alpha is  0.345431358159
beta 1 is  0.000271683611823
beta 2 is  0.0439997994535
MSE is  50.4414739502
ASE is  16651.613061
